#### Imports

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import logging
import time

from project import Dataset, Route
from project.Metaheuristics import Metaheuristic

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
)
logger = logging.getLogger("TCC")

logging.getLogger("MESA.mesa.model").setLevel(logging.WARNING)

BASE_PATH = Path("..")

N_RUNS = 10
MAS_N_STEPS = 10

# Exploring Solomon Benchmark

## Comprehending diferences between datasets

In [ ]:
# type hints for better readability and IDE support
axes: np.ndarray
ax: plt.Axes

paths = [
    BASE_PATH / "Datasets" / "C101.txt",
    BASE_PATH / "Datasets" / "R101.txt",
    BASE_PATH / "Datasets" / "RC101.txt",
    BASE_PATH / "Datasets" / "RC201.txt",
]

datasets = [Dataset(p) for p in paths]

avg_window_times = [
    (ds.customers_df['due_date'] - ds.customers_df['ready_time']).mean()
    for ds in datasets
 ]

for ds, avg_time in zip(datasets, avg_window_times):
    logger.info(f"{ds.name} - Average Time Window: {avg_time:.2f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, ds in zip(axes, datasets):
    ax.scatter(ds.customers_df.x[1:], ds.customers_df.y[1:], c='blue', label='Customers', alpha=0.6, s=10)
    ax.scatter(ds.customers_df.x[0], ds.customers_df.y[0], c='red', marker='s', s=60, label='Depot')
    ax.set_title(ds.name)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.grid(True, linestyle='--', alpha=0.5)

for ax in axes[len(datasets):]:
    ax.axis("off")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")
fig.suptitle("Solomon Instances - Geographic Distribution")
fig.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

## Testing base code

Validations for cost function, vizualization methods

In [ ]:
# Read solution and plot routes with cost
solution_path = BASE_PATH / "Solutions" / "rc201.txt"
dataset_path = BASE_PATH / "Datasets" / "rc201.txt"

ds = Dataset(dataset_path)
route = Route(dataset=ds)
route.read_solution(solution_path)

feasibility = route.is_feasible()
total_cost = route.calc_total_distance()
num_subroutes = route.calc_total_vehicles()
total_wait = route.calc_total_waiting_time()
total_service = route.calc_total_service_time()
total_time = route.calc_total_time()

logger.info(f"Feasibility of the solution: {'Feasible' if feasibility else 'Infeasible'}")
logger.info(f"Number of routes in solution: {num_subroutes}")
logger.info(f"Total distance (cost): {total_cost:.2f}")

if (round(total_cost, 2) == 1406.94) and (num_subroutes == 4) and feasibility:
    logger.info("The total cost matches the expected values from literature.")
    # NOTE: This is a validation check to ensure the solution's cost is as expected
    # Check value from literature here: https://www.sintef.no/projectweb/top/vrptw/100-customers/

route.plot_routes('Benchmark Solution - RC201')

# Metaheuristics 

Creating a runner to encapsulate multiple running on metaheuristic algorithms.

In [ ]:
class MetaheuristicRunner:
    def __init__(self, algo_cls: type, dataset: Dataset, runs: int = N_RUNS, seed_start: int = 42, **algo_kwargs: object) -> None:
        self.algo_cls = algo_cls
        self.dataset = dataset
        self.runs = runs
        self.seed_start = seed_start
        self.algo_kwargs = dict(algo_kwargs)

    def run(self, plot_best: bool = True, best_plot_title: str | None = None) -> dict[str, object]:
        runtimes: list[float] = []
        vehicles: list[int] = []
        distances: list[float] = []
        best_route: Route | None = None
        best_k: int | None = None
        best_d: float | None = None
        best_history_k: list[int] = []
        best_history_d: list[float] = []

        for run_idx in range(self.runs):
            seed = self.seed_start + run_idx
            params = dict(self.algo_kwargs)
            params["seed"] = seed

            algo: Metaheuristic = self.algo_cls(dataset=self.dataset, **params)

            start_time = time.process_time()
            route = algo.solve()
            elapsed = time.process_time() - start_time

            k, d = route.cost_function()
            runtimes.append(elapsed)
            vehicles.append(k)
            distances.append(d)

            if best_route is None or best_k is None or best_d is None or (k, d) < (best_k, best_d):
                best_route = route
                best_k, best_d = k, d
                best_history_k = []
                best_history_d = []
                for historic_route in algo.route_historic:
                    hk, hd = historic_route.cost_function()
                    best_history_k.append(hk)
                    best_history_d.append(hd)

        summary = {
            "runs": self.runs,
            "avg_runtime_sec": float(np.mean(runtimes)),
            "std_runtime_sec": float(np.std(runtimes, ddof=1)) if self.runs > 1 else 0.0,
            "avg_vehicles": float(np.mean(vehicles)),
            "std_vehicles": float(np.std(vehicles, ddof=1)) if self.runs > 1 else 0.0,
            "avg_distance": float(np.mean(distances)),
            "std_distance": float(np.std(distances, ddof=1)) if self.runs > 1 else 0.0,
            "best_vehicles": int(best_k),
            "best_distance": float(best_d),
            "best_route": best_route,
            "best_history_k": best_history_k,
            "best_history_d": best_history_d,
        }

        logger.info(
            f"{self.algo_cls.__name__} over {self.runs} runs | "
            f"avg runtime: {summary['avg_runtime_sec']:.4f}s ± {summary['std_runtime_sec']:.4f} | "
            f"avg K: {summary['avg_vehicles']:.2f} ± {summary['std_vehicles']:.2f} | "
            f"avg D: {summary['avg_distance']:.2f} ± {summary['std_distance']:.2f} | "
            f"best: (K={summary['best_vehicles']}, D={summary['best_distance']:.2f})"
        )

        if plot_best and best_route is not None:
            title = best_plot_title or f"{self.algo_cls.__name__} Best of {self.runs} Runs"
            best_route.plot_routes(title)

            if best_history_k and best_history_d:
                n_iterations = len(best_history_k)
                x = range(1, n_iterations + 1)
                logger.info(f"{self.algo_cls.__name__} best-run iterations (from plot X-axis): {n_iterations}")

                fig, ax1 = plt.subplots(figsize=(6, 3))
                ax1.plot(x, best_history_k, color='tab:blue', label='Vehicles')
                ax1.set_xlabel("Iteration")
                ax1.set_ylabel("Vehicles (K)", color='tab:blue')
                ax1.tick_params(axis='y', labelcolor='tab:blue')

                ax2 = ax1.twinx()
                ax2.plot(x, best_history_d, color='tab:orange', label='Distance')
                ax2.set_ylabel("Distance (D)", color='tab:orange')
                ax2.tick_params(axis='y', labelcolor='tab:orange')

                plt.title(f"{self.algo_cls.__name__} Best Run Cost History")
                fig.tight_layout()
                plt.show()

        return summary

## Simulated Annealing

In [ ]:
from project.Metaheuristics import SimulatedAnnealing

sa_summary = MetaheuristicRunner(
    SimulatedAnnealing,
    ds,
    runs=2,
    seed_start=42,
    initial_temp=1250.0,
    cooling_rate=0.99,
    min_temp=3.0,
    iterations_per_temp=70,
).run(
    plot_best=True,
    best_plot_title=f"Simulated Annealing Best ({N_RUNS} runs)",
)

## Tabu Search

In [ ]:
from project.Metaheuristics import TabuSearch

ts_summary = MetaheuristicRunner(
    TabuSearch,
    ds,
    runs=2,
    seed_start=42,
    max_iterations=750,
    tabu_tenure=20,
    neighbor_samples=60,
).run(
    plot_best=True,
    best_plot_title=f"Tabu Search Best ({N_RUNS} runs)",
)

## Genetic Algorithm

In [ ]:
from project.Metaheuristics import GeneticAlgorithm

ga_summary = MetaheuristicRunner(
    GeneticAlgorithm,
    ds,
    runs=3,
    seed_start=42,
    population_size=60,
    generations=600,
    crossover_rate=0.8,
    mutation_rate=0.1,
).run(
    plot_best=True,
    best_plot_title=f"Genetic Algorithm Best ({N_RUNS} runs)",
)

# Multi Agent System

## Simple MAS flow

In [ ]:
from project.mesa_model import VRPOptimizationModel

runs_x_mas = N_RUNS
steps_mas_multi = 10
plot_best_multi = True

mas_multi_runtimes = []
mas_multi_k = []
mas_multi_d = []
mas_multi_best_route = None
mas_multi_best_pair = None
mas_multi_best_log = []
for run_idx in range(runs_x_mas):
    model = VRPOptimizationModel(ds, pool_size=7, seed=42 + run_idx)
    start_time = time.process_time()

    for _ in range(steps_mas_multi):
        model.step()

    elapsed = time.process_time() - start_time
    mas_multi_runtimes.append(elapsed)

    best_run = model.elite_pool.best()
    if best_run is not None:
        k_run, d_run = best_run.cost_function()
        mas_multi_k.append(k_run)
        mas_multi_d.append(d_run)

        if mas_multi_best_pair is None or (k_run, d_run) < mas_multi_best_pair:
            mas_multi_best_pair = (k_run, d_run)
            mas_multi_best_route = best_run
            mas_multi_best_log.append({
                "run": run_idx + 1,
                "vehicles": int(k_run),
                "distance": float(d_run),
                "runtime_sec": float(elapsed),
            })

logger.info(
    f"MAS multi-run ({runs_x_mas} runs) | "
    f"avg runtime: {np.mean(mas_multi_runtimes):.4f}s ± {np.std(mas_multi_runtimes, ddof=1) if len(mas_multi_runtimes) > 1 else 0.0:.4f} | "
    f"avg K: {np.mean(mas_multi_k):.2f} ± {np.std(mas_multi_k, ddof=1) if len(mas_multi_k) > 1 else 0.0:.2f} | "
    f"avg D: {np.mean(mas_multi_d):.2f} ± {np.std(mas_multi_d, ddof=1) if len(mas_multi_d) > 1 else 0.0:.2f}"
)

if mas_multi_best_log:
    logger.info("MAS best-found log (global improvements):")
    for item in mas_multi_best_log:
        logger.info(
            f"  Run {item['run']:>2} -> K={item['vehicles']}, "
            f"D={item['distance']:.2f}, runtime={item['runtime_sec']:.4f}s"
        )

if plot_best_multi and mas_multi_best_route is not None:
    mas_multi_best_route.plot_routes(f"Mesa MAS Best ({runs_x_mas} runs)")

Single run to verify logs.

In [ ]:
from project.mesa_model import VRPOptimizationModel

start_time = time.process_time()
mesa_model = VRPOptimizationModel(ds, pool_size=7, seed=42)
steps = 10
history_k = []
history_d = []
for _ in range(steps):
    logger.info(f"Mesa MAS Step {_ + 1}/{steps}")
    mesa_model.step()
    best_mesa = mesa_model.elite_pool.best()

    if best_mesa is not None:
        k_mesa, d_mesa = best_mesa.cost_function()
        history_k.append(k_mesa)
        history_d.append(d_mesa)

elapsed_mesa = time.process_time() - start_time
logger.info(f"\nMesa MAS runtime: {elapsed_mesa:.4f} seconds")

best_mesa = mesa_model.elite_pool.best()
if best_mesa is not None:
    k_mesa, d_mesa = best_mesa.cost_function()
    logger.info(f"Mesa MAS best -> vehicles: {k_mesa}, distance: {d_mesa:.2f}")
    best_mesa.plot_routes("Mesa MAS Best")
    x = range(1, len(history_k) + 1)
    fig, ax1 = plt.subplots(figsize=(6, 3))
    ax1.plot(x, history_k, marker='o', color='tab:blue', label='Vehicles')
    ax1.set_xlabel("Step")
    ax1.set_ylabel("Vehicles (K)", color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax2 = ax1.twinx()
    ax2.plot(x, history_d, marker='o', color='tab:orange', label='Distance')
    ax2.set_ylabel("Distance (D)", color='tab:orange')
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    plt.title("Elite Pool Best Cost Function History")
    fig.tight_layout()
    plt.show()

## MAS with RL

In [ ]:
from project.mesa_model_rl import VRPOptimizationModelRL

runs_x_mas_rl = 2
steps_mas_rl_multi = 2
plot_best_multi = True

mas_rl_multi_runtimes = []
mas_rl_multi_k = []
mas_rl_multi_d = []
mas_rl_multi_q_states = []
mas_rl_multi_count_states = []
mas_rl_multi_best_route = None
mas_rl_multi_best_model = None
mas_rl_multi_best_pair = None
mas_rl_multi_best_log = []

for run_idx in range(runs_x_mas_rl):
    logger.info(f"--- MAS RL RUN {run_idx + 1}/{runs_x_mas_rl} ---\n")
    model_rl = VRPOptimizationModelRL(ds, pool_size=7, seed=42 + run_idx)
    start_time = time.process_time()

    for _ in range(steps_mas_rl_multi):
        logger.info(f"MAS RL Step {_ + 1}/{steps_mas_rl_multi}")
        model_rl.step()

    elapsed = time.process_time() - start_time
    mas_rl_multi_runtimes.append(elapsed)
    mas_rl_multi_q_states.append(len(model_rl.q_table))
    mas_rl_multi_count_states.append(len(model_rl.action_count_table))

    best_run = model_rl.elite_pool.best()
    if best_run is not None:
        k_run, d_run = best_run.cost_function()
        mas_rl_multi_k.append(k_run)
        mas_rl_multi_d.append(d_run)

        if mas_rl_multi_best_pair is None or (k_run, d_run) < mas_rl_multi_best_pair:
            mas_rl_multi_best_pair = (k_run, d_run)
            mas_rl_multi_best_route = best_run
            mas_rl_multi_best_model = model_rl
            mas_rl_multi_best_log.append({
                "run": run_idx + 1,
                "vehicles": int(k_run),
                "distance": float(d_run),
                "runtime_sec": float(elapsed),
                "q_states": int(len(model_rl.q_table)),
                "visited_states": int(len(model_rl.action_count_table)),
            })

    logger.info(f"Run {run_idx + 1} summary | runtime: {elapsed:.4f}s | "
        f"K: {k_run if best_run is not None else 'N/A'}, D: {d_run if best_run is not None else 'N/A'} | "
        f"Q states: {len(model_rl.q_table)}, Visited states: {len(model_rl.action_count_table)}\n"
    )

logger.info(
    f"\nMAS RL multi-run ({runs_x_mas_rl} runs) | "
    f"avg runtime: {np.mean(mas_rl_multi_runtimes):.4f}s ± {np.std(mas_rl_multi_runtimes, ddof=1) if len(mas_rl_multi_runtimes) > 1 else 0.0:.4f} | "
    f"avg K: {np.mean(mas_rl_multi_k):.2f} ± {np.std(mas_rl_multi_k, ddof=1) if len(mas_rl_multi_k) > 1 else 0.0:.2f} | "
    f"avg D: {np.mean(mas_rl_multi_d):.2f} ± {np.std(mas_rl_multi_d, ddof=1) if len(mas_rl_multi_d) > 1 else 0.0:.2f}"
)
logger.info(
    f"MAS RL states | avg Q states: {np.mean(mas_rl_multi_q_states):.2f}, "
    f"avg visited states: {np.mean(mas_rl_multi_count_states):.2f}"
)

if mas_rl_multi_best_log:
    logger.info("MAS RL best-found solution (global improvements):")
    best_seed = 42 + mas_rl_multi_best_log[0]['run'] - 1
    for item in mas_rl_multi_best_log:
        logger.info(
            f"  Run {item['run']:>2} (seed: {best_seed}) -> K={item['vehicles']}, D={item['distance']:.2f}, "
            f"runtime={item['runtime_sec']:.4f}s, Q states={item['q_states']}, visited={item['visited_states']}"
        )

if plot_best_multi and mas_rl_multi_best_route is not None:
    mas_rl_multi_best_route.plot_routes(f"MAS RL Best ({runs_x_mas_rl} runs)")

mesa_model_rl_multi = mas_rl_multi_best_model

Single run to verify logs.

In [ ]:
from project.mesa_model_rl import VRPOptimizationModelRL

best_found_seed = 42
if 'mas_rl_multi_best_log' in globals() and mas_rl_multi_best_log:
    best_found_seed = 42 + int(mas_rl_multi_best_log[-1]['run']) - 1

logger.info(f"--- MAS RL SINGLE RUN (seed={best_found_seed}) ---\n")

start_time = time.process_time()
mesa_model_rl = VRPOptimizationModelRL(ds, pool_size=7, seed=best_found_seed)
steps = 5 #MAS_N_STEPS
history_k = []
history_d = []

for step_idx in range(steps):
    logger.info(f"MAS RL Step {step_idx + 1}/{steps}")
    mesa_model_rl.step()
    best_rl = mesa_model_rl.elite_pool.best()
    if best_rl is not None:
        k_rl, d_rl = best_rl.cost_function()
        history_k.append(k_rl)
        history_d.append(d_rl)

elapsed_rl = time.process_time() - start_time
logger.info(f"\nMAS RL runtime: {elapsed_rl:.4f} seconds")

best_rl = mesa_model_rl.elite_pool.best()
if best_rl is not None:
    k_rl, d_rl = best_rl.cost_function()
    logger.info(
        f"Run summary | runtime: {elapsed_rl:.4f}s | "
        f"K: {k_rl}, D: {d_rl:.2f} | "
        f"Q states: {len(mesa_model_rl.q_table)}, Visited states: {len(mesa_model_rl.action_count_table)}\n"
    )
    best_rl.plot_routes("MAS RL Best")
    x = range(1, len(history_k) + 1)
    fig, ax1 = plt.subplots(figsize=(6, 3))
    ax1.plot(x, history_k, marker='o', color='tab:blue', label='Vehicles')
    ax1.set_xlabel("Step")
    ax1.set_ylabel("Vehicles (K)", color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax2 = ax1.twinx()
    ax2.plot(x, history_d, marker='o', color='tab:orange', label='Distance')
    ax2.set_ylabel("Distance (D)", color='tab:orange')
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    plt.title("Elite Pool Best Cost Function History (RL)")
    fig.tight_layout()
    plt.show()
else:
    logger.info(
        f"Run summary | runtime: {elapsed_rl:.4f}s | "
        f"K: N/A, D: N/A | "
        f"Q states: {len(mesa_model_rl.q_table)}, Visited states: {len(mesa_model_rl.action_count_table)}\n"
    )

Q-Table plots

In [ ]:
action_labels = ["SA", "TS", "GA"]
states = sorted(set(mesa_model_rl.q_table.keys()) | set(mesa_model_rl.action_count_table.keys()))

if states:
    q_values = np.array([mesa_model_rl.q_table.get(s, [0.0, 0.0, 0.0]) for s in states], dtype=float)
    count_values = np.array([mesa_model_rl.action_count_table.get(s, [0, 0, 0]) for s in states], dtype=int)

    fig1, ax1 = plt.subplots(figsize=(8, max(3, len(states) * 0.45)))
    im1 = ax1.imshow(q_values, cmap="viridis", aspect="auto")
    ax1.set_xticks(np.arange(len(action_labels)))
    ax1.set_xticklabels(action_labels)
    ax1.set_yticks(np.arange(len(states)))
    ax1.set_yticklabels([f"K={k}, Dbin={d_bin}" for k, d_bin in states])
    ax1.set_xlabel("Action")
    ax1.set_ylabel("State")
    ax1.set_title("Q-Table")

    q_abs_max = np.max(np.abs(q_values)) if q_values.size else 0.0
    q_threshold = q_abs_max * 0.5
    for i in range(q_values.shape[0]):
        for j in range(q_values.shape[1]):
            value = q_values[i, j]
            text_color = "white" if abs(value) > q_threshold else "black"
            ax1.text(j, i, f"{value:.2f}", ha="center", va="center", color=text_color, fontsize=8)

    fig1.colorbar(im1, ax=ax1, label="Q-value")
    fig1.tight_layout()
    plt.show()

    algo_totals = np.sum(count_values, axis=0).astype(int)
    logger.info(
        f"Total algo runs -> SA: {int(algo_totals[0])}, "
        f"TS: {int(algo_totals[1])}, "
        f"GA: {int(algo_totals[2])}"
    )

    fig2, ax2 = plt.subplots(figsize=(8, max(3, len(states) * 0.45)))
    im2 = ax2.imshow(count_values, cmap="Blues", aspect="auto")
    ax2.set_xticks(np.arange(len(action_labels)))
    ax2.set_xticklabels(action_labels)
    ax2.set_yticks(np.arange(len(states)))
    ax2.set_yticklabels([f"K={k}, Dbin={d_bin}" for k, d_bin in states])
    ax2.set_xlabel("Action")
    ax2.set_ylabel("State")
    ax2.set_title("Action Run Counts")

    c_max = np.max(count_values) if count_values.size else 0
    c_threshold = c_max * 0.5
    for i in range(count_values.shape[0]):
        for j in range(count_values.shape[1]):
            count = int(count_values[i, j])
            text_color = "white" if count > c_threshold else "black"
            ax2.text(j, i, f"{count}", ha="center", va="center", color=text_color, fontsize=8)

    fig2.colorbar(im2, ax=ax2, label="Runs")
    fig2.tight_layout()
    plt.show()
else:
    logger.info("No states available. Run Cell 21 first.")

# Appendix

Understanding the size of a NP problem.

Estimates the time required to solve a 100-customer TSP instance using exhaustive search (Brute Force).

In [ ]:
import math

num_customers = 100
n_permuted = num_customers - 1 # 1 depot, so we permute (n-1) customers

total_solutions = math.factorial(n_permuted)

solutions_per_ms = 1_000_000_000
solutions_per_sec = solutions_per_ms * 1_000

sec_per_year = 60 * 60 * 24 * 365.25
solutions_per_year = solutions_per_sec * sec_per_year

estimated_years = total_solutions / solutions_per_year

# Contextual comparison: Estimated age of the universe (13.8 billion years)
age_of_universe = 13.8e9
ratio_universe_age = estimated_years / age_of_universe


print(f"\nESTIMATED TIME TO FIND THE OPTIMAL SOLUTION:")
print(f"{estimated_years:.4e} years")

print(f"\nCOMPARISON WITH COSMIC SCALE:")
print(f"This is approximately {ratio_universe_age:.4e} times the age of the universe.")
